**Clone repository. First install git lfs. Pulling all files may take a while**

In [1]:
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/gchronis/features_in_context/
!pip install transformers
%cd features_in_context
!git lfs pull --include=trained_models/*
%cd notebooks

[sudo] password for gsc685: 


sudo: timed out reading password
sudo: a password is required
Git LFS initialized.
Cloning into 'features_in_context'...
remote: Enumerating objects: 682, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 682 (delta 34), reused 60 (delta 27), pack-reused 590 (from 1)
Receiving objects: 100% (682/682), 11.69 MiB | 27.08 MiB/s, done.
Resolving deltas: 100% (411/411), done.
Filtering content: 100% (2/2), 1.34 GiB | 64.46 MiB/s, done.
/home/gsc685/partisan_features_3_23/features_in_context
zsh:1: no matches found: --include=trained_models/*
/home/gsc685/partisan_features_3_23/features_in_context/notebooks


In [ ]:
import sys
import torch
sys.path.append("..")
from src.bert import *

bert = BERTBase()
#buchanan = torch.load('../trained_models/model.plsr.buchanan.allbuthomonyms.5k.300components.500max_iters')
binder = torch.load('../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize300')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

<ipython-input-2-e3be6cc3aeb9>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  binder = torch.load('../trained_models/model.ffnn.binder.5k.50epochs.0.5dropout.lr1e-4.hsize3

In [ ]:
from google.colab import files
import pandas as pd
import io

uploaded = files.upload()

# Reading a CSV file

uploaded_df = pd.read_csv(io.BytesIO(uploaded['one_word_illegal_sentences.csv']))
print(uploaded_df)

Saving one_word_illegal_sentences.csv to one_word_illegal_sentences.csv
            word  cluster                                           sentence
0     immigrants        1  “every local government has to figure out what...
1     immigrants        1  many undocumented immigrants were being booked...
2     immigrants        1  nava offered three examples of undocumented im...
3     immigrants        0  jerry brown is expected to sign the california...
4     immigrants        0  the bill also gives protections to inmates who...
...          ...      ...                                                ...
4117  immigrants        0  to explain, last february a federal trial judg...
4118  immigrants        4  pelosi also questioned why the dream act, whic...
4119  immigrants        0  that hes going back to his priority of deporti...
4120  immigrants        0  yes, we need more coordination between ice and...
4121  immigrants        0  for instance, he has recently revoked daca [de...

[41

In [ ]:
def get_features(row, model = binder, bert = bert):
  feature_map = model.feature_norms.feature_map
  feature_labels = [str(feature_map.get_object(i)) for i in range(0, len(feature_map))]
  predictions = model.predict_in_context(row["word"], row["sentence"], bert)

  #create dictionary where key is feature name and value is feature value
  myDict = {feature_labels[i]: pred for i, pred in enumerate(predictions)}
  #sort
  sortedDict = {key: value for key, value in sorted(myDict.items())}

  return sortedDict

In [ ]:
print(get_features(uploaded_df.iloc[0]))

{'Angry': 1.5682364, 'Arousal': 2.236509, 'Attention': 2.6828074, 'Audition': 2.4541018, 'Away': 0.3612062, 'Benefit': 1.0136997, 'Biomotion': 4.175252, 'Body': 4.6778464, 'Bright': -0.14479415, 'Cognition': -0.11238415, 'Color': 0.20555384, 'Communication': -0.068840325, 'Consequential': 2.2566988, 'Dark': 0.53523856, 'Disgusted': 1.2865187, 'Drive': 1.0446521, 'Duration': 1.4444237, 'Face': 4.4596715, 'Fast': 0.99475145, 'Fearful': 2.1370533, 'Happy': 0.39881307, 'Harm': 2.504361, 'Head': 1.5340761, 'High': 0.90500325, 'Human': 4.0144906, 'Landmark': 1.1017774, 'Large': 2.195234, 'Long': 1.154228, 'Loud': 1.8050085, 'Low': 0.7093204, 'LowerLimb': 1.0611144, 'Motion': 2.0626755, 'Music': 0.9494502, 'Near': -0.42363575, 'Needs': 0.14858638, 'Number': 1.621296, 'Pain': 1.7864196, 'Path': 0.6129387, 'Pattern': 0.30398342, 'Pleasant': 0.5541424, 'Sad': 1.8618796, 'Scene': 2.1644652, 'Self': 0.026364889, 'Shape': 1.4317725, 'Short': 0.5198219, 'Slow': 0.83086675, 'Small': 0.14884154, 'Smel

In [ ]:
data = {'word': [],
        'sentence': [],
        'cluster': [],
        'feature': [],
        'value': []}
for i in range(len(uploaded_df)):
    try:
        feature_dict = get_features(uploaded_df.iloc[i])
        for feature, value in feature_dict.items():
            data['word'].append(uploaded_df.iloc[i]["word"])
            data['sentence'].append(uploaded_df.iloc[i]["sentence"])
            data['cluster'].append(uploaded_df.iloc[i]["cluster"])
            data['feature'].append(feature)
            data['value'].append(value)
    except Exception as e:
        print(f"Error processing row {i}: {e}")

print(data)

Error processing row 1787: list index out of range
Buffered data was truncated after reaching the output size limit.

In [ ]:
df_with_features = pd.DataFrame.from_dict(data)
print(df_with_features)

            word                                           sentence  cluster  \
0       thinking  By not considering these interests explicitly ...        0   
1       thinking  By not considering these interests explicitly ...        0   
2       thinking  By not considering these interests explicitly ...        0   
3       thinking  By not considering these interests explicitly ...        0   
4       thinking  By not considering these interests explicitly ...        0   
...          ...                                                ...      ...   
246817      fire                           There are fire hazards .        4   
246818      fire                           There are fire hazards .        4   
246819      fire                           There are fire hazards .        4   
246820      fire                           There are fire hazards .        4   
246821      fire                           There are fire hazards .        4   

           feature     value  
0       

In [ ]:
df_with_features.to_csv("aliens_and_immigrants (with features).csv", index = False)
files.download("aliens_and_immigrants (with features).csv (with features).csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>